In [0]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def find_text(text, start_marker, end_marker):
    result = re.search(start_marker + "(.*)" + end_marker, text)
    return result.group(1)


def get_pages(section_name, section_number, url_prefix):

    # List of pages to be pulled
    df_pages = pd.DataFrame(None, columns = ['section_name', 
                                             'section_number', 
                                             'url', 
                                             'thread_title'])
    # Retrieve section links
    f = requests.get(url_prefix)

    # Look up value for last page in data-last=
    data_last_page = int(find_text(f.text,'data-last="', '"' ))
    if data_last_page < 25:
        end_page = data_last_page
    else:
        end_page = int(data_last_page/2)
    
    # Threads prefix
    threads_prefix = "http://www.ultimatemetal.com/forum/"
    
    for i in range(end_page):
        print(section_name + " loop " + str(i) + " out of " + str(end_page))
    
        link = url_prefix + "page-" + str(i)
        f = requests.get(link)
        
        soup = BeautifulSoup(f.text, "html.parser")
        headers = soup.select('h3.title')
        
        for j in range(len(headers)):
            #print("going through headers " + str(j))
            
            # Get end of each thread link
            url_suffix = find_text(str(headers[j]), 'href="', '/"')
            url = threads_prefix + url_suffix 
            thread_title = headers[j].text.strip()
            
            # Get page to append to data set
            df_pages_temp = pd.DataFrame({"section_name": [section_name],
                                          "section_number": [section_number],
                                          "url": [url],
                                          "thread_title": [thread_title]})
                                          
            df_pages = df_pages.append(df_pages_temp)
            
    return df_pages
    

In [0]:

df_pages_main = get_pages('Main', 
                              0,
                              "http://www.ultimatemetal.com/forum/forums/andy-sneap/")

df_pages_backstage = get_pages('Backstage', 
                              1,
                              "http://www.ultimatemetal.com/forum/forums/andy-sneap-backstage/")

df_pages_foh = get_pages('FOH', 
                              2,
                              "http://www.ultimatemetal.com/forum/forums/andy-sneap-foh/")

df_pages_practice_room = get_pages('Practice_Room', 
                              3,
                              "http://www.ultimatemetal.com/forum/forums/andy-sneap-practice-room/")

df_pages_backline = get_pages('Backline', 
                              4,
                              "http://www.ultimatemetal.com/forum/forums/andy-sneap-backline/")

df_pages_merch_stand = get_pages('Merch_Stand', 
                              5,
                              "http://www.ultimatemetal.com/forum/forums/andy-sneap-merch-stand/")

df_pages_bar = get_pages('Bar', 
                              6,
                              "http://www.ultimatemetal.com/forum/forums/andy-sneap-bar/")


Main loop 0 out of 368
Main loop 1 out of 368
Main loop 2 out of 368
Main loop 3 out of 368
Main loop 4 out of 368
Main loop 5 out of 368
Main loop 6 out of 368
Main loop 7 out of 368
Main loop 8 out of 368
Main loop 9 out of 368
Main loop 10 out of 368
Main loop 11 out of 368
Main loop 12 out of 368
Main loop 13 out of 368
Main loop 14 out of 368
Main loop 15 out of 368
Main loop 16 out of 368
Main loop 17 out of 368
Main loop 18 out of 368
Main loop 19 out of 368
Main loop 20 out of 368
Main loop 21 out of 368
Main loop 22 out of 368
Main loop 23 out of 368
Main loop 24 out of 368
Main loop 25 out of 368
Main loop 26 out of 368
Main loop 27 out of 368
Main loop 28 out of 368
Main loop 29 out of 368
Main loop 30 out of 368
Main loop 31 out of 368
Main loop 32 out of 368
Main loop 33 out of 368
Main loop 34 out of 368
Main loop 35 out of 368
Main loop 36 out of 368
Main loop 37 out of 368
Main loop 38 out of 368
Main loop 39 out of 368
Main loop 40 out of 368
Main loop 41 out of 368
Ma

In [0]:

# Combine all forum data together
df_pages_all = pd.concat([df_pages_main,
                          df_pages_backstage,
                          df_pages_foh,
                          df_pages_practice_room,
                          df_pages_backline,
                          df_pages_merch_stand,
                          df_pages_bar], ignore_index=True)

# Create column to capture html
df_pages_all['initial_message_text'] = None
    
iteration_counter = 0
# Get initial message for all pages
for idx, url in enumerate(df_pages_all.url):

    if idx < iteration_counter:
        continue
    else:
        iteration_counter = iteration_counter + 1
        
    f = requests.get(url)
        
    soup = BeautifulSoup(f.text, "html.parser")
    initial_message = soup.select('div.messageContent')
    
    if initial_message == []:
        continue
    
    df_pages_all.initial_message_text[idx] = initial_message[0].text
    
    # Backup every 1000
    if idx % 1000 == 0:
        print(idx, url)
        df_pages_all.to_pickle("/content/gdrive/My Drive/df_pages_all_backup.pkl")


